In [1]:
import pandas as pd
df = pd.read_csv("Inventory Items.csv")

In [2]:
# STEP 3: Define Category Mappings
category_keywords = {
    "meals": [
        "meal", "dinner", "soup", "turkey", "hot", "chili", "plate", "prepared", "sandwich", "kitchen",
        "quiche", "pot pie", "ham", "steak", "drumstick", "sausage", "frozen"
    ],
    "pantry": [
        "canned", "beans", "rice", "grain", "vegetable", "fruit", "donation", "non-perishable", "dry goods",
        "tuna", "apple", "pepper", "pasta", "sauce", "v8", "bloody mary", "dessert",
        "yogurt", "eggs", "juice", "milk", "nuts", "snack", "granola", "protein", 
        "gatorade", "soda", "pretzels", "cookie", "chips", "butter", "cheese",
        "cereal", "cracker", "bar", "popcorn", "drink", "beverage", "oats",
        "onion", "potato", "ginger", "bread", "bun", "coffee", "creamer", "half and half", "tea"
    ],
    "hygiene": [
        "soap", "toothbrush", "shampoo", "kit", "sanitary", "deodorant", "toothpaste", "body wash", "basic needs",
        "bathroom", "toilet", "hand sanitizer", "razor", "q-tip", "lip balm", "conditioner", "paper towel", "napkin"
    ],
    "clothing": [
        "shirt", "jacket", "sock", "pants", "coat", "clothing", "sweater", "blanket"
    ],
    "medical": [
        "syringe", "naloxone", "first aid", "bandage", "gloves", "test kit",
        "ssdi", "ssi", "snap", "medicare", "medicaid", "husky", "insurance", "medical assistance"
    ],
    "case_management": [
        "vital document", "section 8", "dmv", "id", "birth certificate", "social security", "homeless",
        "housing support", "phone", "assurance", "safelink", "spdat", "verification", "benefit enrollment"
    ],
    "harm_reduction": [
        "meth", "prep pad", "ziplock", "pipe", "straw", "foil", "chore boy", "lube", "screen", "razor blade",
        "teaspoon", "mouth piece", "q-tip", "chopstick", "cotton balls", "vitamin c powder"
    ],
    "pet_care": [
        "dog food", "cat food", "pet"
    ],
    "donated_goods": [  # New for things like toys, stuffed animals, etc.
        "toy", "stuffed animal", "non-food", "miscellaneous"
    ]
}

# STEP 4: Define Category-to-Code Mapping
category_code_map = {
    "meals":     ("020", "1"),
    "pantry":    ("030", "4"),
    "hygiene":   ("040", "2"),
    "clothing":  ("010", "6"),
    "medical":   ("040", "5"),
    "case_management": ("050", "7")
}

# Add new category mappings
category_code_map.update({
    "harm_reduction": ("040", "9"),
    "pet_care": ("060", "8"),
    "donated_goods": ("070", "3")
})

# STEP 5: Program and Building Maps
program_map = {
    "drop in": "1",
    "outreach": "2",
    "pantry": "3"
}

building_map = {
    "266": "266",  # Main
    "260": "260",  # Storage
    "151": "151"   # Admin
}


In [3]:
# STEP 6: Item Number Generation Logic
# =======================
def generate_category_based_item_number(row):
    name = str(row.get('Name', '')).lower()
    type_ = str(row.get('Type', '')).lower()
    tags = str(row.get('Tags', '')).lower()
    number = str(row.get('Number', '')).lower()

    # Category → Department + ItemType
    dept_code = "000"
    item_type_code = "0"
    for cat, keywords in category_keywords.items():
        if any(kw in name or kw in type_ or kw in tags for kw in keywords):
            dept_code, item_type_code = category_code_map.get(cat, ("000", "0"))
            break

    # Program
    prog_code = "0"
    for keyword, code in program_map.items():
        if keyword in name or keyword in tags:
            prog_code = code
            break

    # Building
    bldg_code = "000"
    for bcode in building_map:
        if number.startswith(bcode) or bcode in tags:
            bldg_code = building_map[bcode]
            break

    return f"{dept_code}{prog_code}.{item_type_code}-{bldg_code}"


In [4]:

# )=====================================
# STEP 5: Apply to your DataFrame
# =====================================
df['FormattedItemNumber'] = df.apply(generate_category_based_item_number, axis=1)



In [5]:
# =====================================
# STEP 6: Save to Excel for future edits
# =====================================
df.to_excel("cleaning_workbook.xlsx", index=False)


In [6]:
df

,Number,Type,Name,Override Rules,Unit,Value/ Unit,Weight/ Unit,Tags,Groups,Status,Shows On,FormattedItemNumber
0,266-1,DESK Dinners,Servery at 266 State,NaN,Bag,0.0,0.0,"266 - Meals, Current",NaN,Active,"Visit,Kiosk,",0200.1-266
1,266-2,DESK Dinners,DESK Outreach Meals,NaN,Bag,0.0,0.0,MEALS,NaN,Active,"Visit,",0202.1-266
2,266-3,DESK Dinners,Chapel on the Green Meal Service,NaN,Each,0.0,0.0,MEALS,NaN,Active,"Visit,Kiosk,Store,",0200.1-266
3,266-4,DESK Dinners,Trinity Church on the Green Meal Service,NaN,Each,0.0,0.0,MEALS,NaN,Active,"Visit,Kiosk,Store,",0200.1-266
4,266-5,DESK Dinners,New Haven Green Meal Service,NaN,Each,0.0,0.0,MEALS,NaN,Active,"Visit,Kiosk,Store,",0200.1-266
...,...,...,...,...,...,...,...,...,...,...,...,...
969,33020252,Ready to Bake,Beans Northern Dried,NaN,Bag,0.0,0.0,NaN,Beans Northern Dried Case,Active,"Visit,Kiosk,Store,",0300.4-000
970,33020253,Ready to Bake,Sea Salt Caramel Dessert Sauce,NaN,Bag,0.0,0.0,NaN,Sea Salt Caramel Dessert Sauce Case,Active,"Visit,Kiosk,Store,",0300.4-000
971,33020254,Ready to Bake,V8 Bloody Mary,NaN,Bag,0.0,0.0,NaN,V8 Bloody Mary Case,Active,"Visit,Kiosk,Store,",0300.4-000
972,33020255,Ready to Bake,Pasta Elbow,NaN,Bag,0.0,0.0,NaN,Pasta Elbow Case,Active,"Visit,Kiosk,Store,",0300.4-000


In [7]:
unmatched = df[df['FormattedItemNumber'].str.startswith("0000")]
print(f"\n❗ Unmatched items (start with '0000'): {len(unmatched)}")
print(unmatched[['Name', 'Type', 'Number', 'FormattedItemNumber']].head(100))



❗ Unmatched items (start with '0000'): 47
                                                 Name  \
152                                              Bags   
155                                       Paper Goods   
163                                             Candy   
174                          Tender Mercies - Regular   
176                        Half & Half - Shelf Stable   
184                                Grocery - Assorted   
213                                 Water - Sparkling   
215                                 Water - Sparkling   
228                                Grocery - Assorted   
233                                           Noodles   
245                                              Baby   
247                                              Bags   
250                                       Paper Goods   
256                        Half & Half - Shelf Stable   
258                          Tender Mercies - Regular   
268                                         P

In [8]:
unmatched.to_excel(r"E:\pantry\needs_manual_review.xlsx", index=False)
print("📄 Saved: needs_manual_review.xlsx")

📄 Saved: needs_manual_review.xlsx
